In [51]:
%pylab inline
import pandas as pd
import csv
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
import string

Populating the interactive namespace from numpy and matplotlib


In [52]:
#read csv
amazon_df = pd.read_csv('/media/backup/Data/Amazon/amazon_male.csv', sep='|',encoding='utf8',quoting=csv.QUOTE_NONE, dtype={'Overall_Rating':float, 'Timestamp': int})

In [53]:
#arranging row index
amazon_df.index = amazon_df.index+1

In [54]:
amazon_df.head()

,Reviewer_Id,Product_Id,Name,Gender,Helpfulness,Review,Overall_Rating,Timestamp
1,A00000262KYZUE4J55XGL,B003UYU16G,Steven N Elich,male,"0,0",It is and does exactly what the description sa...,5.0,1353456000
2,A00000922W28P2OCH6JSE,B000VEBG9Y,Gabriel Merrill,male,"0,0",Very mobile product. Efficient. Easy to use; h...,3.0,1395619200
3,A00000922W28P2OCH6JSE,B001EJMS6K,Gabriel Merrill,male,"0,0",Easy to use a mobile. If you're taller than 4f...,4.0,1395619200
4,A00000922W28P2OCH6JSE,B003XJCNVO,Gabriel Merrill,male,"0,0",Love this feeder. Heavy duty & capacity. Best ...,4.0,1395619200
5,A00000922W28P2OCH6JSE,B00G3KN9LI,Gabriel Merrill,male,"0,0","Solid, stable mount. Holds iPhone with phone p...",4.0,1396051200


In [55]:
#amazon_df.Review.to_csv(Reviews.csv)
amazon_df.shape

(16950602, 8)

In [56]:
amazon_df.drop(['Reviewer_Id','Product_Id', 'Name', 'Helpfulness', 'Overall_Rating', 'Timestamp'], axis=1, inplace=True)    

In [57]:
amazon_df = amazon_df.iloc[:50000]

In [58]:
amazon_df.head()

,Gender,Review
1,male,It is and does exactly what the description sa...
2,male,Very mobile product. Efficient. Easy to use; h...
3,male,Easy to use a mobile. If you're taller than 4f...
4,male,Love this feeder. Heavy duty & capacity. Best ...
5,male,"Solid, stable mount. Holds iPhone with phone p..."


# Female Data

In [59]:
amazon_df_f = pd.read_csv('/media/backup/Data/Amazon/amazon_female.csv', sep='|',encoding='utf8',quoting=csv.QUOTE_NONE, dtype={'Overall_Rating':float, 'Timestamp': int})

In [60]:
#arranging row index
amazon_df_f.index = amazon_df_f.index+1

In [61]:
amazon_df_f.head()

,Reviewer_Id,Product_Id,Name,Gender,Helpfulness,Review,Overall_Rating,Timestamp
1,A00001362Q1PGIX2FYSSH,B00125OS3C,Pamela Bellamy,female,"0,0",Beautiful photos/film with wonderful music. G...,5.0,1377907200
2,A00001362Q1PGIX2FYSSH,B002IGHX40,Pamela Bellamy,female,"1,1",My idea of Colorado is &#34;Mountains&#34;. C...,2.0,1377907200
3,A00003262KNLZOSMMMFVV,B002Y2U8MC,Harue Rojas,female,"1,1","It is not a sticker, it is a Chritsmas story ...",5.0,1361145600
4,A00003262KNLZOSMMMFVV,B004P598FY,Harue Rojas,female,"1,1","LOve the size and the details, and its very co...",5.0,1361145600
5,A00003262KNLZOSMMMFVV,B005MU3UE6,Harue Rojas,female,"1,1",Its very colorful and the image once you put...,5.0,1361145600


In [62]:
amazon_df_f.drop(['Reviewer_Id','Product_Id', 'Name', 'Helpfulness', 'Overall_Rating', 'Timestamp'], axis=1, inplace=True)    

In [63]:
amazon_df_f = amazon_df_f.iloc[:50000]

In [64]:
amazon_df_f.shape

(50000, 2)

In [65]:
amazon_df_f.head()

,Gender,Review
1,female,Beautiful photos/film with wonderful music. G...
2,female,My idea of Colorado is &#34;Mountains&#34;. C...
3,female,"It is not a sticker, it is a Chritsmas story ..."
4,female,"LOve the size and the details, and its very co..."
5,female,Its very colorful and the image once you put...


In [66]:
data = amazon_df.append(amazon_df_f, ignore_index= True)

In [67]:
data.index = data.index + 1

In [68]:
data.shape

(100000, 2)

In [69]:
data = data.sample(frac = 1).reset_index(drop=True)

In [70]:
data.index = data.index +1

In [71]:
data.head()

,Gender,Review
1,male,Item was exactly as described. Not thick wire...
2,female,This is perfect for me. I don't have the stre...
3,male,I just just received my twelve cans of this st...
4,female,"Excellent, in my opinion one of the best brand..."
5,female,"To believe I didn't want to read this book , s..."


# Data Preprocessing

In [72]:
#data preprocessing
porter_stemmer = nltk.PorterStemmer()
words = stopwords.words("english")

In [73]:
data['Review'] = data['Review'].apply(str)

In [74]:
#remove punctuation for each word
#maketrans() method returns a translation table that maps each character in the 
#intab string into the character at the same position in the outtab string
table = str.maketrans('','', string.punctuation)

In [75]:
#data cleaning and forming a new column
#remove words that are not alphanumeric eg. remove s of what's 

data['Cleaned_Reviews'] = data['Review'].apply(lambda x: " ".join([i.translate(table) for i in x.split() if i.isalpha() if i not in words]).lower())

In [76]:
data.head()

,Gender,Review,Cleaned_Reviews
1,male,Item was exactly as described. Not thick wire...,item exactly not thick wire casing i knew i bo...
2,female,This is perfect for me. I don't have the stre...,this perfect i strength pull this easy handle ...
3,male,I just just received my twelve cans of this st...,i received twelve cans good cans good expirati...
4,female,"Excellent, in my opinion one of the best brand...",opinion one best brands hard drives good price...
5,female,"To believe I didn't want to read this book , s...",to believe i want read book this book absolute...


In [77]:
#1 for female and 0 for male
data['Gender_conv'] = pd.get_dummies(data.Gender)['female']

In [78]:
data.head()

,Gender,Review,Cleaned_Reviews,Gender_conv
1,male,Item was exactly as described. Not thick wire...,item exactly not thick wire casing i knew i bo...,0
2,female,This is perfect for me. I don't have the stre...,this perfect i strength pull this easy handle ...,1
3,male,I just just received my twelve cans of this st...,i received twelve cans good cans good expirati...,0
4,female,"Excellent, in my opinion one of the best brand...",opinion one best brands hard drives good price...,1
5,female,"To believe I didn't want to read this book , s...",to believe i want read book this book absolute...,1


In [79]:
#improvements
#stemming the reviews

data['Cleaned_Reviews'] = data['Cleaned_Reviews'].apply(lambda x: " ".join([porter_stemmer.stem(word) for word in x.split()]))

In [80]:
data.head()

,Gender,Review,Cleaned_Reviews,Gender_conv
1,male,Item was exactly as described. Not thick wire...,item exactli not thick wire case i knew i boug...,0
2,female,This is perfect for me. I don't have the stre...,thi perfect i strength pull thi easi handl gre...,1
3,male,I just just received my twelve cans of this st...,i receiv twelv can good can good expir date tw...,0
4,female,"Excellent, in my opinion one of the best brand...",opinion one best brand hard drive good price c...,1
5,female,"To believe I didn't want to read this book , s...",to believ i want read book thi book absolut am...,1


In [81]:
data['Word_Count'] = data['Cleaned_Reviews'].str.split().str.len()

In [82]:
data.head()

,Gender,Review,Cleaned_Reviews,Gender_conv,Word_Count
1,male,Item was exactly as described. Not thick wire...,item exactli not thick wire case i knew i boug...,0,14
2,female,This is perfect for me. I don't have the stre...,thi perfect i strength pull thi easi handl gre...,1,19
3,male,I just just received my twelve cans of this st...,i receiv twelv can good can good expir date tw...,0,33
4,female,"Excellent, in my opinion one of the best brand...",opinion one best brand hard drive good price c...,1,87
5,female,"To believe I didn't want to read this book , s...",to believ i want read book thi book absolut am...,1,32


In [83]:
X = data[['Cleaned_Reviews', 'Word_Count']]
Y = data['Gender_conv']

#test train split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 0)

In [84]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
    
    
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]

In [85]:
classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('Cleaned_Reviews')),
            ('tfidf', TfidfVectorizer( stop_words="english",
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            #('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])),
        ('words', Pipeline([
            ('wordext', NumberSelector('Word_Count')),
            ('wscaler', StandardScaler()),
        ])),
    ])),
    ('clf', LogisticRegression(C=1.0, penalty='l2', max_iter=3000, dual=False)),
#    ('clf', RandomForestClassifier()),
    ])

In [86]:
classifier.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('colext', TextSelector(field='Cleaned_Reviews')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', inp...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [87]:
preds = classifier.predict(X_test)

In [89]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds))
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))

# [[ true_negatives   false_positives]
#  [ false_negatives  true_positives]]

Accuracy: 0.68535
Precision: 0.6814896607107007
             precision    recall  f1-score   support

          0       0.69      0.69      0.69     10091
          1       0.68      0.69      0.68      9909

avg / total       0.69      0.69      0.69     20000

[[6918 3173]
 [3120 6789]]
